In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
from scipy.stats import pearsonr
import statsmodels.api as sm
import statsmodels.regression.linear_model as linear_model
from statsmodels.stats.outliers_influence import variance_inflation_factor



In [3]:
power_stations_raw = pd.read_csv('Data/power_stations_clean.csv', delimiter=';', encoding='latin1' )

power_stations_raw.head()


,MaStR-Nr. der Stromerzeugungseinheit,Anlagenbetreiber,Anzeige-Name der Stromerzeugungseinheit,PLZ der Einheit,Ort der Einheit,Straße der Einheit,Hausnummer der Einheit,Bundesland der Einheit,Datum der erstmaligen Inbetriebnahme der Einheit,Datum der endgültigen Stilllegung der Einheit (Datum/Jahr),...,Auswertung Energieträger,Wärmeauskopplung (KWK)\n(ja/nein),Ist die Stromerzeugungseinheit ein Bestandteil eines Grenzkraftwerkes?,Bruttoleistung in MW,Nettonennleistung (elektrische Wirkleistung) in MW,Ist die Stromerzeugungseinheit ein Bestandteil eines Grenzkraftwerkes?: ja \nNettonennleistung der Einspeisung in ein deutsches Netz:,Technologie der Stromerzeugung,Volleinspeisung oder Teileinspeisung?,Anschlussnetzbetreiber,Spannungsebene
0,SEE915851127786,Stadtwerke Aachen AG,BHKW Schwarzer Weg,52070,Aachen,Schwarzer Weg,19 b,Nordrhein-Westfalen,05.04.2023,NaN,...,Erdgas,Ja,nein,"22,54","21,51",NaN,Verbrennungsmotor,Volleinspeisung,Regionetz GmbH (SNB911641710114),Mittelspannung
1,SEE946736241334,Palm Power GmbH & Co. KG,HKW Aalen,73432,Aalen,Palm Allee,1,Baden-Württemberg,03.11.2021,NaN,...,Erdgas,Ja,nein,"79,19","78,30",NaN,Gasturbinen mit nachgeschalteter Dampfturbine,Volleinspeisung,Netze BW GmbH (SNB948311994307),Hochspannung; Hochspannung
2,SEE930596800480,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT A,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,22.12.1990,NaN,...,Erdgas,Nein,nein,"38,34","37,10",NaN,Gasturbinen mit Abhitzekessel,Volleinspeisung,50Hertz Transmission GmbH (SNB982046657236),Hochspannung
3,SEE929797382345,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT B,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,10.12.1990,NaN,...,Erdgas,Nein,nein,"38,24","37,10",NaN,Gasturbinen mit Abhitzekessel,Volleinspeisung,50Hertz Transmission GmbH (SNB982046657236),Hochspannung
4,SEE988046628214,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT C,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,26.03.1991,NaN,...,Erdgas,Nein,nein,"38,34","37,10",NaN,Gasturbinen mit Abhitzekessel,Volleinspeisung,50Hertz Transmission GmbH (SNB982046657236),Hochspannung


In [36]:
key = list(power_stations_raw.columns)

In [37]:
len(key)

23

In [38]:
key

['MaStR-Nr. der Stromerzeugungseinheit',
 'Anlagenbetreiber',
 'Anzeige-Name der Stromerzeugungseinheit',
 'PLZ der Einheit',
 'Ort der Einheit',
 'Straße der Einheit',
 'Hausnummer der Einheit',
 'Bundesland der Einheit',
 'Datum der erstmaligen Inbetriebnahme der Einheit',
 'Datum der endgültigen Stilllegung der Einheit (Datum/Jahr)',
 'Kraftwerksstatus der Einheit',
 'Energieträger',
 'Hauptbrennstoff',
 'Auswertung Energieträger',
 'Wärmeauskopplung (KWK)\n(ja/nein)',
 'Ist die Stromerzeugungseinheit ein Bestandteil eines Grenzkraftwerkes?',
 'Bruttoleistung in MW',
 'Nettonennleistung (elektrische Wirkleistung) in MW',
 'Ist die Stromerzeugungseinheit ein Bestandteil eines Grenzkraftwerkes?: ja \nNettonennleistung der Einspeisung in ein deutsches Netz:',
 'Technologie der Stromerzeugung',
 'Volleinspeisung oder Teileinspeisung?',
 'Anschlussnetzbetreiber',
 'Spannungsebene']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2287 entries, 0 to 2286
Data columns (total 23 columns):
 #   Column                                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                                --------------  ----- 
 0   MaStR-Nr. der Stromerzeugungseinheit                                                                                                  2287 non-null   object
 1   Anlagenbetreiber                                                                                                                      2025 non-null   object
 2   Anzeige-Name der Stromerzeugungseinheit                                                                                               2120 non-null   object
 3   PLZ der Einheit                                                               

In [43]:
power_station_clean = power_stations_raw.fillna('-')

power_station_clean.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2287 entries, 0 to 2286
Data columns (total 23 columns):
 #   Column                                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                                --------------  ----- 
 0   MaStR-Nr. der Stromerzeugungseinheit                                                                                                  2287 non-null   object
 1   Anlagenbetreiber                                                                                                                      2287 non-null   object
 2   Anzeige-Name der Stromerzeugungseinheit                                                                                               2287 non-null   object
 3   PLZ der Einheit                                                               

In [59]:
power_station_clean[power_station_clean['Bruttoleistung in MW']=='-'].index

Index([2015, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031,
       ...
       2133, 2134, 2135, 2136, 2137, 2138, 2139, 2141, 2142, 2145],
      dtype='int64', length=119)

In [55]:
power_station_clean = power_station_clean.drop(power_station_clean.loc[power_station_clean['Bruttoleistung in MW']=='-'].index)

KeyError: "['MaStR-Nr. der Stromerzeugungseinheit', 'Anlagenbetreiber', 'Anzeige-Name der Stromerzeugungseinheit', 'PLZ der Einheit', 'Ort der Einheit', 'Straße der Einheit', 'Hausnummer der Einheit', 'Bundesland der Einheit', 'Datum der erstmaligen Inbetriebnahme der Einheit', 'Datum der endgültigen Stilllegung der Einheit (Datum/Jahr)', 'Kraftwerksstatus der Einheit', 'Energieträger', 'Hauptbrennstoff', 'Auswertung Energieträger', 'Wärmeauskopplung (KWK)\\n(ja/nein)', 'Ist die Stromerzeugungseinheit ein Bestandteil eines Grenzkraftwerkes?', 'Bruttoleistung in MW', 'Nettonennleistung (elektrische Wirkleistung) in MW', 'Ist die Stromerzeugungseinheit ein Bestandteil eines Grenzkraftwerkes?: ja \\nNettonennleistung der Einspeisung in ein deutsches Netz:', 'Technologie der Stromerzeugung', 'Volleinspeisung oder Teileinspeisung?', 'Anschlussnetzbetreiber', 'Spannungsebene'] not found in axis"

In [22]:
values = {
    
}
new_values = []
for i in values:
    if pd.isna(i):
        new_values.append('-')
    else:
        new_values.append(i) 


power_stations_raw.iloc[:,9] = new_values



pd.filna(power_stations_raw)


,MaStR-Nr. der Stromerzeugungseinheit,Anlagenbetreiber,Anzeige-Name der Stromerzeugungseinheit,PLZ der Einheit,Ort der Einheit,Straße der Einheit,Hausnummer der Einheit,Bundesland der Einheit,Datum der erstmaligen Inbetriebnahme der Einheit,Datum der endgültigen Stilllegung der Einheit (Datum/Jahr),...,Auswertung Energieträger,Wärmeauskopplung (KWK)\n(ja/nein),Ist die Stromerzeugungseinheit ein Bestandteil eines Grenzkraftwerkes?,Bruttoleistung in MW,Nettonennleistung (elektrische Wirkleistung) in MW,Ist die Stromerzeugungseinheit ein Bestandteil eines Grenzkraftwerkes?: ja \nNettonennleistung der Einspeisung in ein deutsches Netz:,Technologie der Stromerzeugung,Volleinspeisung oder Teileinspeisung?,Anschlussnetzbetreiber,Spannungsebene
0,SEE915851127786,Stadtwerke Aachen AG,BHKW Schwarzer Weg,52070,Aachen,Schwarzer Weg,19 b,Nordrhein-Westfalen,05.04.2023,-,...,Erdgas,Ja,nein,"22,54","21,51",NaN,Verbrennungsmotor,Volleinspeisung,Regionetz GmbH (SNB911641710114),Mittelspannung
1,SEE946736241334,Palm Power GmbH & Co. KG,HKW Aalen,73432,Aalen,Palm Allee,1,Baden-Württemberg,03.11.2021,-,...,Erdgas,Ja,nein,"79,19","78,30",NaN,Gasturbinen mit nachgeschalteter Dampfturbine,Volleinspeisung,Netze BW GmbH (SNB948311994307),Hochspannung; Hochspannung
2,SEE930596800480,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT A,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,22.12.1990,-,...,Erdgas,Nein,nein,"38,34","37,10",NaN,Gasturbinen mit Abhitzekessel,Volleinspeisung,50Hertz Transmission GmbH (SNB982046657236),Hochspannung
3,SEE929797382345,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT B,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,10.12.1990,-,...,Erdgas,Nein,nein,"38,24","37,10",NaN,Gasturbinen mit Abhitzekessel,Volleinspeisung,50Hertz Transmission GmbH (SNB982046657236),Hochspannung
4,SEE988046628214,Lausitz Energie Kraftwerke AG,Gasturbinenkraftwerk Ahrensfelde GT C,16356,Ahrensfelde,Lindenberger Straße,12,Brandenburg,26.03.1991,-,...,Erdgas,Nein,nein,"38,34","37,10",NaN,Gasturbinen mit Abhitzekessel,Volleinspeisung,50Hertz Transmission GmbH (SNB982046657236),Hochspannung
